<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/crawler/Investors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import re

# 01: kospi, 02: kosdaq, 03: futures
# date: today
# page = 10 days
date = str(datetime.datetime.now().date().strftime("%Y%m%d"))
market = '01'
url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=' + date + '&sosok=' + market + '&page=1'
result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
bs_obj = BeautifulSoup(result.content, "html.parser")
last_page = int(bs_obj.find_all('td', {'class':'pgRR'})[0].find_all('a')[0]['href'].split('=')[-1])
columns = ['날짜', '개인', '외국인', '기관계','금융투자', '보험', '투신(사모)', '은행', '기타금융기관', '연기금등', '기타법인']

def get_investors(market='01', page='1'):
  date = str(datetime.datetime.now().date().strftime("%Y%m%d"))
  url = 'https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate='+ date +'&sosok='+ market +'&page=' + str(page)
  result = requests.get(url, headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'})
  bs_obj = BeautifulSoup(result.content, "html.parser")
  date = [x.get_text() for x in bs_obj.find_all('td', {'class':re.compile(r'date2')})]
  tds = [x.get_text() for x in bs_obj.find_all('td', {'class':re.compile(r'rate*')})]
  tds = np.reshape(tds, (10,-1))

  date = [datetime.datetime.strptime(x, '%y.%m.%d') for x in date]
  data = tds.tolist()
  [x.insert(0, date[data.index(x)]) for x in data]
  df = pd.DataFrame(columns = columns, data=data)
  return df

In [2]:
import multiprocessing

with multiprocessing.Pool(processes=30) as pool:
    dfs = pool.starmap(get_investors, [['01', x] for x in range(1,last_page)])

In [3]:
investors_df = pd.DataFrame()
for df in dfs:
  investors_df = pd.concat([investors_df,df], ignore_index=True)

In [4]:
investors_df

,날짜,개인,외국인,기관계,금융투자,보험,투신(사모),은행,기타금융기관,연기금등,기타법인
0,2021-10-08,"4,809",-920,"-4,124","-3,354",-123,14,2,28,-691,234
1,2021-10-07,"-5,317","-1,071","6,143","5,823",-74,580,19,-7,-199,244
2,2021-10-06,"1,718","-2,760",834,889,-22,404,-7,-48,-382,208
3,2021-10-05,"3,530","-6,232","2,362","3,067",-167,-164,-48,33,-360,340
4,2021-10-01,"7,609","-3,000","-4,873","-3,610",-157,"-1,031",27,175,-277,264
...,...,...,...,...,...,...,...,...,...,...,...
4135,2005-01-18,-434,"1,047",-613,97,-42,-825,-11,74,93,0
4136,2005-01-17,"-2,578","3,273",-695,-387,147,-755,-31,204,128,0
4137,2005-01-14,"-3,914","1,892","2,022",-133,-134,"1,661",91,117,419,0
4138,2005-01-13,-533,-269,802,-628,-7,404,67,54,912,0
